## imports ##

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import os, sys
import seaborn as sns
from scipy.stats import pearsonr
import mdtraj as md 
import numpy as np 
import os, sys 

In [3]:
def source_module(module_file: str, local_module_name: str = None):
    import importlib
    """to add a module from a user defined python script into the local name space"""
    if local_module_name is None:
        local_module_name = module_file.split("/")[-1].replace(".py", "")
    if len(module_file.split("/")) == 1 or module_file.split("/")[-2] == ".":
        module_dir = os.getcwd()
    else:
        module_dir = "/".join(module_file.split("/")[:-1])
    sys.path.insert(0, module_dir)
    module = importlib.import_module(module_file.split("/")[-1].replace(".py", ""))
    g = globals()
    g[local_module_name] = module
    pass

# importing a python script where I keep a lot of the basic functions I use!
source_module("/Users/anjalidhar/Desktop/Packages/mdtools.py")
source_module("/Users/anjalidhar/Desktop/Packages/Writhe_Package/package/visualize/visualize.py")

# from visualize import palettes

### Loading trajectories ###

In [4]:
trajs = {}
runs = ['41', '47'] # '23'
tdir = '/Users/anjalidhar/Desktop/MD/trajectories'
for run in runs: 
    traj_dir = f'{tdir}/{run}_all'
    traj_files = mdtools.lsdir(traj_dir, keyword='.xtc')
    pdb = mdtools.lsdir(traj_dir, keyword='pdb')[0]
    trajs[run] = [md.load(file, top=pdb) for file in traj_files]


In [5]:
traj_dir = f'{tdir}/23_all'
traj_files = mdtools.lsdir(traj_dir, keyword='.dcd')
pdb = mdtools.lsdir(traj_dir, keyword='pdb')[0]
trajs['23'] = [md.load(file, top=pdb) for file in traj_files]

### Now, trying to find the bound frames... ###

In [6]:
bound_fraction = {}
runs = ['41', '47', '23']
for run in runs: 
    trajectories = trajs[run]
    bound_fraction[run] = []
    for t in trajectories: 
        bin_arr = mdtools.residue_distances(t, np.arange(20), np.array([21]))
        contacts = np.where(np.sum(bin_arr, axis=1) > 0)[0].shape[0]
        bound_fraction[run].append(contacts / bin_arr.shape[0])
        

In [7]:
for run in runs: 
    print(f'Avg BF for {run}: {np.mean(np.array(bound_fraction[run]))}')

Avg BF for 41: 0.7426728970820597
Avg BF for 47: 0.833832828158166
Avg BF for 23: 0.5236053621884648
